### Redes Neurais Multilayer Perceptron

Prevendo diagnóstico de diabete com redes neurais multicamada.


https://www.kaggle.com/datasets/mathchi/diabetes-data-set?resource=download

**Contexto**

Este conjunto de dados é originalmente do Instituto Nacional de Diabetes e Doenças Digestivas e Renais. O objetivo é prever com base em medidas diagnósticas se um paciente tem diabetes. Em particular, todos os pacientes aqui são **mulheres** com pelo menos 21 anos de ascendência indígena Pima.

**Features:**

**Gravidezes:** Número de vezes que engravidou

**Glicose:** Concentração de glicose plasmática a 2 horas em um teste oral de tolerância à glicose

**Pressão arterial:** pressão arterial diastólica (mm Hg)

**Espessura da pele:** espessura da dobra cutânea do tríceps (mm)

**Insulina:** insulina sérica de 2 horas (mu U/ml)

**IMC:** Índice de massa corporal (peso em kg/(altura em m)^2)

**DiabetesPedigreeFunction:** função que pontua a probabilidade de diabetes com base na história familiar

**Idade:** Idade (anos)

**Resultado**: variável de classe (0 ou 1)

Instalando pacotes de redes neurais

In [ ]:
#! pip install keras
#! pip install tensorflow

In [ ]:
from sklearn.model_selection import train_test_split   #carregando a biblioteca para separação de base de treino e teste
from sklearn.preprocessing import StandardScaler       #Normalização dos dados

import pandas as pd                                    #manipulação de dados
import matplotlib.pyplot as plt                        #plot de gráficos
import seaborn as sns                                  #plot de gráficos
import numpy as np

# Keras para rede neural
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf


In [ ]:
#Subindo a base dos dados:

df = pd.read_csv('diabetes.csv')

In [ ]:
#Analisando o total de linhas e colunas do dataset
df.shape

In [ ]:
df.head()

In [ ]:
#Analisando informações estatísticas do dataset
df.describe().T

### Pré-processamento dos dados - Data Clean

In [ ]:
#Analisando valores nulos:

df.isnull().sum()

### Conhecendo nossos dados

In [ ]:
#Gráfico do tipo boxplot podemos analisar a distribuição dos dados e suas medidas centrais, como por exemplo a média, mediana, os outliers e pontos discrepantes.
df.plot(kind='box', subplots=True, layout=(4,4), figsize=(15, 15))
plt.show()

In [ ]:
#Potando histogramas para analisar a simetria dos dados
df.hist(bins=100, figsize=(12, 12))
plt.show()

### Analisando as correlações das variáveis

In [ ]:
correlation_matrix = df.corr().round(2)

fig, ax = plt.subplots(figsize=(8,5))    
sns.heatmap(data=correlation_matrix, annot=True, linewidths=.5, ax=ax)

### Vamos iniciar nossa rede neural multilayer Perceptron!

Separando os dados em treino e teste.

In [ ]:
X = df[['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age']] #variáveis explicativas
y = df['Outcome'] #variável target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) #separando os dados

**Padronizando os dados**

Nessa etapa de padronização é realizada a normalização dos dados. As redes neurais trabalham melhor quando temos dados normalizados em uma mesma escala.

A função StandardScaler é usada para dimensionar os dados. O dimensionamento dos dados resultaria em uma convergência mais rápida para o valor ótimo global para funções de otimização da função de perda. Aqui usamos a classe StandardScaler do Sklearn que realiza a normalização do z-score. A normalização do z-score subtrai cada dado de sua média e o divide pelo desvio padrão dos dados. Vamos treinar com o conjunto de dados dimensionado.

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

### Executando nossa rede neural de multicamdas

**Configuração da rede:**

In [ ]:
tf.random.set_seed(7)

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=8)) 
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax')) #sigmoid, tanh, relu, softmax

# Compile the model
model.compile(optimizer='adam', #adam, sgd
              loss='mean_squared_error', #mean_squared_error, sparse_categorical_crossentropy
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=50)

In [ ]:
from tqdm.keras import TqdmCallback

# model fit params
epoch = 100

# fit the keras model on the dataset
hist = model.fit(X_train
                  ,y_train
                  ,epochs = epoch
                  ,shuffle=True
                  ,validation_data=(X_test, y_test)
                  ,verbose=0
                  ,callbacks=[TqdmCallback(verbose=0)]
          )

# save model
#model.save('DiabetesClassificação')

In [ ]:
# visualize training results
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs_range = range(epoch)

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Acurácia de Treinamento')
plt.plot(epochs_range, val_acc, label='Acurácia de Validação')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Erro de treinamento')
plt.plot(epochs_range, val_loss, label='Erro de Validação')
plt.legend(loc='upper right')
plt.title('Erro de treinamento vs validação')
plt.show()

### Realizando as previsões

In [ ]:
pred_train= model.predict(X_train) #previsão realizada com os dados de treino
pred_test= model.predict(X_test)   #previsão realizada com os dados de teste

In [ ]:
# Resultado da previsão com dados de treino
scores = model.evaluate(X_train, y_train)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   

In [ ]:
# Resultado da previsão com dados de teste
scores2 = model.evaluate(X_test, y_test)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    